In [1]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [2]:
from data import DATA_ROOT as DATA_PATH

In [3]:
import glob 

files = all_files = glob.glob(f"{DATA_PATH}/temp_train/*.csv")

In [4]:
from pyspark.sql.session import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Distributed Word Embeddings")\
        .getOrCreate()

In [5]:
df = spark.read.options(header=True, maxColumns=22500).csv(files)

```python
from pyspark.sql import functions as F

for col in df.columns:
    if col != "actual_words":
        print(type(df.select(F.sum(col)).collect()[0][0]))
        break
```

In [6]:
!pip install joblib
!pip install joblibspark

In [11]:
type(df.columns[0])

str

In [ ]:
import multiprocessing
from pyspark.sql import functions as F
from joblib import Parallel, delayed


threshold = 100
num_cores = multiprocessing.cpu_count() - 1


def get_columns_below_threshold(col):
    if col != "actual_words":
        col_sum = df.select(F.sum(col)).collect()[0][0]
        print(col, col_cum)
        if col_sum < threshold:
            return col
        else:
            return ""
        
        
# columns = Parallel(n_jobs=num_cores)(
#     delayed(get_columns_below_threshold)(col) for col in df.columns
# )
columns = [get_columns_below_threshold(col) for col in df.columns]

In [ ]:
columns = list(filter(None, columns))
df = df.drop(*columns)
df.write.csv(f"{DATA_PATH}/embeddings_with_sum_of_columns_below_{threshold}.csv") ## df is an existing DataFrame object.